In [3]:
# ============================================================
# 01 - Importación de librerías necesarias
# ============================================================
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
import csv, os, re, time, sys, json


# ============================================================
# 02 - Cargar configuración JSON (credenciales y URLs)
# ============================================================
# El archivo ConfigWBUSA.json contiene el usuario, contraseña y URLs de login.
CONFIG_PATH = "ConfigWBUSA.json"

if not os.path.isfile(CONFIG_PATH):
    raise FileNotFoundError(f"[ERROR] No se encontró el archivo de configuración {CONFIG_PATH}")

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

USER = config["USER"]
PWD  = config["PWD"]
LOGIN_URL = config["LOGIN_URL"]
CSPS_URL  = config["CSPS_URL"]


# ============================================================
# 03 - Configuración de archivos de entrada y salida
# ============================================================
INPUT_CSV  = "../InOutWebScrap/InUSAWebScrap/input_parts.csv"   # CSV con columna PartNumber
OUTPUT_DIR = "../InOutWebScrap/OutUSAWebScrap"
OUTPUT_CSV = os.path.join(OUTPUT_DIR, "parts_OT60_OT61.csv")

# Orden de campos preferidos en la tabla de resultados
PREFERRED_FIELDS_ORDER = [
    "Customer Price", "List Price", "Discount", "Core Value", "Freight",
    "Warehouse", "Backorder", "Lead Time", "Currency", "UOM"
]


# ============================================================
# 04 - Inicialización del navegador Chrome con Selenium
# ============================================================
options = Options()
options.add_argument("--start-maximized")  # Abrir ventana maximizada
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 30)


# ============================================================
# 05 - Funciones utilitarias (helpers)
# ============================================================
def clean_text(s: str) -> str:
    """Limpia espacios raros y convierte el texto a una sola línea legible."""
    return " ".join((s or "").replace("\xa0", " ").replace("&nbsp;", " ").split()).strip()

def to_number(text: str):
    """Convierte un string numérico en float (maneja comas/puntos)."""
    t = clean_text(text)
    if not t:
        return None
    m = re.findall(r"[-+]?\d[\d,]*\.?\d*", t)
    if not m:
        return None
    try:
        return float(m[0].replace(",", ""))
    except Exception:
        return None

def safe_find(by, selector, timeout=30):
    """Busca un elemento asegurando que exista en el DOM antes de devolverlo."""
    return WebDriverWait(driver, timeout).until(EC.presence_of_element_located((by, selector)))

def safe_click(by, selector, timeout=30):
    """Hace clic en un elemento asegurando que esté disponible."""
    el = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((by, selector)))
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    el.click()
    return el

def switch_to_frame_by_name(name, timeout=30):
    """Cambia al frame por su nombre (ej. 'menu', 'detail')."""
    WebDriverWait(driver, timeout).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, name)))

def back_to_default():
    """Vuelve al contexto principal fuera de los frames."""
    driver.switch_to.default_content()

def reenter_detail_frame():
    """Regresa al frame 'detail' después de que se recarga la página."""
    back_to_default()
    switch_to_frame_by_name("detail", 30)
    safe_find(By.TAG_NAME, "body", 30)


# ============================================================
# 06 - Funciones específicas del flujo
# ============================================================
def sum_disponibilidad():
    """Suma los valores de 'Available Qty' en la tabla de disponibilidad."""
    total = 0.0
    try:
        table = driver.find_element(By.XPATH, "//table[.//td[normalize-space()='Available Qty']]")
    except Exception:
        return 0.0

    rows = table.find_elements(By.XPATH, ".//tr[td]")
    for r in rows:
        tds = r.find_elements(By.TAG_NAME, "td")
        if len(tds) >= 2:
            num = to_number(clean_text(tds[1].text))
            if num is not None:
                total += num
    return total

def open_general_part_inquiry():
    """Abre el menú de 'General Part Inquiry' dentro del portal CSPS."""
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "frameset")))
    switch_to_frame_by_name("menu", 20)
    actions = ActionChains(driver)

    try:
        expand_icon = safe_click(By.ID, "folderIcon14", 10)
        actions.move_to_element(expand_icon).pause(0.2).perform()
    except Exception:
        pass

    link = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@title='General Part Inquiry']"))
    )
    actions.move_to_element(link).pause(0.2).click().perform()

    back_to_default()
    switch_to_frame_by_name("detail", 20)
    safe_find(By.ID, "partNr", 20)

def do_login_and_open_csps():
    """Realiza login y abre CSPS en nueva pestaña, luego entra a General Part Inquiry."""
    driver.get(LOGIN_URL)
    safe_find(By.ID, "userID", 20).send_keys(USER)
    driver.find_element(By.ID, "password").send_keys(PWD)
    driver.find_element(By.ID, "btn-submit").click()
    wait.until(EC.url_contains("my.dlrportal.com"))
    driver.switch_to.new_window('tab')
    driver.get(CSPS_URL)
    open_general_part_inquiry()

def select_order_type(ord_type_value: str):
    """Selecciona el tipo de orden (Order Type) con reintentos por si falla."""
    for attempt in range(3):
        try:
            cmb = safe_find(By.ID, "cmbOrdType", 20)
            Select(cmb).select_by_value(ord_type_value)
            driver.execute_script("arguments[0].dispatchEvent(new Event('change', {bubbles:true}));", cmb)
            return
        except (StaleElementReferenceException, TimeoutException):
            reenter_detail_frame()
    cmb = safe_find(By.ID, "cmbOrdType", 10)
    Select(cmb).select_by_value(ord_type_value)

def click_price_inquiry():
    """Hace clic en el botón 'Price Inquiry' y espera que cargue la página."""
    price_btn = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//input[@type='button' and @value='Price Inquiry']"))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", price_btn)
    try:
        price_btn.click()
    except Exception:
        driver.execute_script("priceInquiry();")
    reenter_detail_frame()

def read_selected_order_table() -> dict:
    """Lee los datos de la tabla 'Selected Order' y los devuelve en un diccionario."""
    table = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, "//table[.//td[normalize-space()='Selected Order']]"))
    )
    rows = table.find_elements(By.XPATH, ".//tr[position()>1]")

    data = {}
    for tr in rows:
        tds = tr.find_elements(By.TAG_NAME, "td")
        if len(tds) >= 4:
            field = clean_text(tds[0].text)
            val   = clean_text(tds[3].text)
            if field and val and val != "-":
                data[field] = val
    return data

def fetch_part_for_ordertype(part_number: str, ord_type_value: str) -> dict:
    """Consulta un número de parte para un tipo de orden y devuelve sus datos."""
    input_part = safe_find(By.ID, "partNr", 20)
    input_part.clear()
    input_part.send_keys(part_number)
    safe_click(By.ID, "displayBtn", 20)

    reenter_detail_frame()
    select_order_type(ord_type_value)
    click_price_inquiry()

    try:
        hdr = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//*[normalize-space()='Available Qty']"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", hdr)
        time.sleep(0.2)
    except Exception:
        pass

    table_data = read_selected_order_table()
    stock_total = int(sum_disponibilidad() or 0)
    result = {"AvailabilityTotal": stock_total}
    result.update(table_data)
    return result

def compose_headers_from_results(results_by_ord):
    """Crea la lista de columnas del CSV final (ej. OT60_Customer Price, OT61_Customer Price)."""
    all_fields = set()
    for d in results_by_ord.values():
        all_fields.update(d.keys())
    fields = ["AvailabilityTotal"]
    for pf in PREFERRED_FIELDS_ORDER:
        if pf in all_fields and pf not in fields:
            fields.append(pf)
    for f in sorted(all_fields):
        if f not in fields:
            fields.append(f)
    headers = []
    for ot in sorted(results_by_ord.keys()):
        for f in fields:
            headers.append(f"OT{ot}_{f}")
    return headers, fields


# ============================================================
# 07 - Main: flujo principal
# ============================================================
try:
    if not os.path.isfile(INPUT_CSV):
        print(f"[ERROR] No se encuentra {INPUT_CSV}.")
        sys.exit(1)

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Login y acceso al módulo
    do_login_and_open_csps()

    # Leer partes del CSV
    parts = []
    with open(INPUT_CSV, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        if "PartNumber" not in reader.fieldnames:
            print("[ERROR] El CSV de entrada debe tener columna 'PartNumber'.")
            sys.exit(1)
        for row in reader:
            p = clean_text(row.get("PartNumber", ""))
            if p:
                parts.append(p)

    if not parts:
        print("[WARN] No hay PartNumber en el CSV de entrada.")
        sys.exit(0)

    # Consultar primera parte (para armar headers)
    preview_part = parts[0]
    results_preview = {ot: fetch_part_for_ordertype(preview_part, ot) for ot in ("60", "61")}
    headers, ordered_fields = compose_headers_from_results(results_preview)
    headers = ["PartNumber"] + headers

    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=headers)
        writer.writeheader()

        # Escribir primera parte
        row_out = {"PartNumber": preview_part}
        for ot in ("60", "61"):
            for fld in ordered_fields:
                row_out[f"OT{ot}_{fld}"] = results_preview[ot].get(fld, "")
        writer.writerow(row_out)
        print(f"[OK] {preview_part} exportado (OT60/OT61).")

        # Procesar el resto
        for part in parts[1:]:
            try:
                results = {ot: fetch_part_for_ordertype(part, ot) for ot in ("60", "61")}
                row = {"PartNumber": part}
                for ot in ("60", "61"):
                    for fld in ordered_fields:
                        row[f"OT{ot}_{fld}"] = results[ot].get(fld, "")
                writer.writerow(row)
                print(f"[OK] {part} exportado (OT60/OT61).")
            except Exception as ex:
                print(f"[ERROR] Procesando {part}: {repr(ex)}")

    print(f"[OK] CSV final generado: {OUTPUT_CSV}")

except Exception as e:
    print("[ERROR]", repr(e))

finally:
    try:
        input("Pulsa ENTER para cerrar...")
    except Exception:
        pass
    driver.quit()


[OK] 5801781949 exportado (OT60/OT61).
[OK] 47133709 exportado (OT60/OT61).
[OK] 504127327 exportado (OT60/OT61).
[OK] 5801781935 exportado (OT60/OT61).
[OK] 500042644 exportado (OT60/OT61).
[OK] 500042643 exportado (OT60/OT61).
[OK] 5801781949 exportado (OT60/OT61).
[OK] 504092109 exportado (OT60/OT61).
[OK] 48133712 exportado (OT60/OT61).
[OK] 5802486932 exportado (OT60/OT61).
[OK] 84455696 exportado (OT60/OT61).
[OK] 98473706 exportado (OT60/OT61).
[OK] 1930181 exportado (OT60/OT61).
[OK] CSV final generado: ../InOutWebScrap/OutUSAWebScrap\parts_OT60_OT61.csv
